# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/alexi/Desktop/WeatherPy/python-api-challenge/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-21.35,99,100,2.52,RU,1709683754
1,1,margaret river,-33.9500,115.0667,14.89,76,5,5.69,AU,1709683774
2,2,ushuaia,-54.8000,-68.3000,9.44,95,94,2.43,AR,1709683752
3,3,chonchi,-42.6128,-73.8092,14.56,89,60,1.86,CL,1709683774
4,4,shawinigan,46.5668,-72.7491,7.90,96,100,0.55,CA,1709684131


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,blackmans bay,-43.0167,147.3167,22.51,51,0,0.89,AU,1709683764
60,60,cabo san lucas,22.8909,-109.9124,25.90,67,0,2.06,MX,1709684238
117,117,mount gambier,-37.8333,140.7667,24.43,39,0,1.86,AU,1709684372
255,255,umluj,25.0213,37.2685,21.88,54,0,3.97,SA,1709684416
350,350,moree,-29.4667,149.8500,26.15,48,0,2.84,AU,1709684112
444,444,new norfolk,-42.7826,147.0587,23.17,36,0,2.13,AU,1709683919
529,529,raver,21.2500,76.0333,21.08,23,0,1.50,IN,1709684509
540,540,rabigh,22.7986,39.0349,22.84,53,0,2.16,SA,1709684513


### Step 3: Create a new DataFrame called `hotel_df`.

In [53]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,blackmans bay,AU,-43.0167,147.3167,51,
60,cabo san lucas,MX,22.8909,-109.9124,67,
117,mount gambier,AU,-37.8333,140.7667,39,
255,umluj,SA,25.0213,37.2685,54,
350,moree,AU,-29.4667,149.8500,48,
444,new norfolk,AU,-42.7826,147.0587,36,
529,raver,IN,21.2500,76.0333,23,
540,rabigh,SA,22.7986,39.0349,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
cabo san lucas - nearest hotel: Comfort Rooms
mount gambier - nearest hotel: The Old Mount Gambier Gaol
umluj - nearest hotel: No hotel found
moree - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
raver - nearest hotel: No hotel found
rabigh - nearest hotel: S.A. TALKE Villa


,City,Country,Lat,Lng,Humidity,Hotel Name
32,blackmans bay,AU,-43.0167,147.3167,51,Villa Howden
60,cabo san lucas,MX,22.8909,-109.9124,67,Comfort Rooms
117,mount gambier,AU,-37.8333,140.7667,39,The Old Mount Gambier Gaol
255,umluj,SA,25.0213,37.2685,54,No hotel found
350,moree,AU,-29.4667,149.8500,48,No hotel found
444,new norfolk,AU,-42.7826,147.0587,36,Woodbridge on the Derwent
529,raver,IN,21.2500,76.0333,23,No hotel found
540,rabigh,SA,22.7986,39.0349,53,S.A. TALKE Villa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)